In [2]:
!pip install jsonapi_client

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
inputBioms = ["root:Host-associated:Human:Digestive system:Oral:Subgingival plaque","root:Host-associated:Human:Digestive system:Oral:Supragingival plaque"]

biomNameMapping =  {
    "root:Host-associated:Human:Digestive system:Oral:Subgingival plaque": "biom1",
    "root:Host-associated:Human:Digestive system:Oral:Supragingival plaque": "biom2"
}

runName = "run_01"

max_sample_count = 500

In [4]:
# Imports
!pip install conorm

from jsonapi_client import Session
import pandas as pd
import requests
import os
import numpy as np
import conorm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [150]:


###################
# Definie Functions
###################

########################################################
# input: biom string from MGnify
# https://www.ebi.ac.uk/metagenomics/browse/biomes/
# example: GetBiomSamplesByIds(root:Engineered:Bioreactor)
#
#
# output: df with the first <max_sample_count> 
########################################################
def GetBiomSamplesByIds(biom_id: str):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        i = 0
        biomes_dfs = []
        for r in mgnify.iterate(f'biomes/{biom_id}/samples'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
            i += 1
            if(i == max_sample_count):
                break
    main_biomes_df = pd.concat(biomes_dfs)
    return main_biomes_df

########################################################
# input: study ACCESSION from mgnify
# https://www.ebi.ac.uk/metagenomics/browse/studies
# example: GetBiomSamplesByIds(MGYS00006539)
#
#
# output: list with 2 objects
# when output[0] == None, we have a study without any analysis data
# when output[0] == 1,  output[1] = df with all meta-analysis Informations (accession, date, version, ...) for each sample in the study
########################################################

def GetAnalysisFromStudy(_study_id):
    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        biomes_dfs = []
        for r in mgnify.iterate(f'studies/{_study_id}/analyses'):
            biom_df = pd.json_normalize(r.json)
            biom_df['url'] = str(r.links.self)
            biomes_dfs.append(biom_df)
    # testing if study has no analysis data
    if(biomes_dfs == []):
        return [None,None]
    main_biomes_df = pd.concat(biomes_dfs)

    return [1,main_biomes_df]

In [151]:

for biom in inputBioms:
    df_samples = GetBiomSamplesByIds(biom)

    sample_accessions = (df_samples['attributes.accession'].to_list())

    studies = df_samples['relationships.studies.data'].to_list()
    ####################
    # for each of the studies we test if they have analysis data.
    # if so we save the data in <study_analaysises>,
    # until we have data for every sample.
    ####################

    # list of df with analysis data from studys
    study_analaysises = []
    # study-accessions from studys without analysis
    studies_without_analysis = []
    # study-accessions from studys having analysis data in <study_analaysises>
    studies_Ids = []

    for study in studies:
        # list of studies for sample
        s_ids_sample = []
        for s in str(study).split("'"):
            if('MGYS' in s):
                s_ids_sample.append(s) 
        # for each study
        for st in s_ids_sample:
            # test if we havent checked study
            if(st in studies_Ids):
                # we allready have analysis data for this study
                break
            if(st not in studies_without_analysis):
                # Get analysis data for this study
                study_analaysis = GetAnalysisFromStudy(st)
                if(study_analaysis[0] == None):
                    # discard if study has no analysis
                    studies_without_analysis.append(st)
                    continue
                # keep if study has analysis
                studies_Ids.append(st)
                study_analaysises.append(study_analaysis[1])
                break
    # Merge all analysis data into one large df
    study_analaysis = pd.concat(study_analaysises)
    analysis_accesions = []
    for sample_acc in sample_accessions:
        acc = study_analaysis[study_analaysis['relationships.sample.data.id'] == sample_acc]['attributes.accession'].to_list()
        analysis_accesions.append([len(acc),acc])
    analysis_accesions

    ##################################
    # for each of of our samples we try to get the taxonomic assingments from MGnify
    # if none of the analysis accessions of a sample return a file, we skip the sample
    # (this happens when for e.g. the version of the MGnify pipeline is to old)
    #
    # Trying for "OTUs and taxonomic assignments for SSU rRNA" and "All reads encoding SSU rRNA" beacuse of different pipeline versions
    #
    # all Analysis files are saved in 'outputs/collection', so that they are automatically exported into Galaxy as one Collection
    #
    # analysis files are saved under 'outputs/collection/<analysis-accession>-<sample-accession>.tsv'
    ##################################

    # for each of our samples
    for analysis_sample in analysis_accesions:
        url = ""
        # for each analysis-accession for the sample
        for analysis_accession in analysis_sample[1]:
            print(f"start grab for {analysis_accession}")
            
            # Get all downloads for one analysis
            with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
                
                dfs = []
                for r in mgnify.iterate(f'analyses/{analysis_accession}/downloads'):
                    df = pd.json_normalize(r.json)
                    df['url'] = str(r.links.self)
                    dfs.append(df)
                
            main_df = pd.concat(dfs)

            data_type = "TSV"
            data_label = "OTUs and taxonomic assignments for SSU rRNA"
            
            # get URL for data_label = "OTUs and taxonomic assignments for SSU rRNA"
            c1 = main_df["attributes.file-format.name"] == data_type
            c2 = main_df["attributes.description.description"] == data_label
            if(main_df.loc[(c1 & c2), "url"].size == 0):
                # if we dont get a match try the same with data_label = 'All reads encoding SSU rRNA'
                data_label = "All reads encoding SSU rRNA"
                c2 = main_df["attributes.description.description"] == data_label
                if(main_df.loc[(c1 & c2), "url"].size == 0):
                    continue
                else:
                    url = main_df.loc[(c1 & c2), "url"].iloc[0]
                    break
            url = main_df.loc[(c1 & c2), "url"].iloc[0]
            break

        if(url == ""):
            print(f'no analysis found for {analysis_sample}')
            continue

        ##########################
        # download the data
        #########################

        data_output_folder = f'outputs/collection/{runName}/{biomNameMapping[biom]}'
        if(not os.path.exists(data_output_folder)):
            os.makedirs(data_output_folder, exist_ok=True)

        response = requests.get(url)

        if not response:
            print(f"Could not download file, got response: {response.status_code}")
            break
            
        print(url)
        data_output_path = os.path.join(data_output_folder, f"{analysis_accession}-{study_analaysis[study_analaysis['attributes.accession'] == analysis_accession]['relationships.sample.data.id'].to_list()[0]}.tsv")
        with open(data_output_path, "w") as f:
            f.write(response.text)

start grab for MGYA00661793
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661793/file/ERZ14207391_FASTA_SSU_OTU.tsv
start grab for MGYA00661792
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661792/file/ERZ14207393_FASTA_SSU_OTU.tsv
start grab for MGYA00661791
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661791/file/ERZ14207392_FASTA_SSU_OTU.tsv
start grab for MGYA00661789
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661789/file/ERZ14207394_FASTA_SSU_OTU.tsv
start grab for MGYA00661787
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661787/file/ERZ14207366_FASTA_SSU_OTU.tsv
start grab for MGYA00661785
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661785/file/ERZ14207367_FASTA_SSU_OTU.tsv
start grab for MGYA00661784
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661784/file/ERZ14207368_FASTA_SSU_OTU.tsv
start grab for MGYA00661783
https://www.ebi.ac.uk/metagenomics/api/v1/analyses/MGYA00661783/file/ERZ14207369_FA

In [152]:
from functools import reduce

def MergeOnTaxaLevelFunction(taxa, runName,biomName):
    if(not os.path.isdir(f"./outputs/collection/{runName}/merged/")):
        os.mkdir(f"./outputs/collection/{runName}/merged/")
    galaxyInput_rankToMerge = taxa
    # selection from "superkingdom" "kingdom" "phylum" "class" "order" "family" "genus" "species" "all" "only counts"

    galaxyInput_countTables = runName


    mappingFromTaxaToNumber = {
        "superkingdom":1,
        "kingdom":2,
        "phylum":3,
        "class":4,
        "order":5,
        "family":6,
        "genus":7,
        "species":8,
        "all" : 10,         # to prevent key error in converting rank to number
        "only counts" : 10  # only counts and all will be handled when creating the tables
    }

    rankToMerge = mappingFromTaxaToNumber[galaxyInput_rankToMerge]

    filePaths = os.listdir(f"./outputs/collection/{runName}/{biomName}")

    #filePaths = []

    

    tables = []
    for file in filePaths:
        # Read out all Tables and remove unneccecary informations
        df = pd.read_csv(f"./outputs/collection/{runName}/{biomName}/{file}",sep="\t",header=1)
        df = df.iloc[: , 1:]
        if("taxid" in df.columns):
            df = df.drop(columns=["taxid"])
        df.columns = [file.split("/")[-1].replace(".tsv",""),"#KEY"]
        columns = df.columns.tolist()
        columns.reverse()
        df = df[columns]
        tables.append(df)
    # Merge all tables into one
    df = reduce(lambda df1,df2:pd.merge(df1,df2,on="#KEY",how="outer"),tables)
    df = df.fillna(0)
    if(galaxyInput_rankToMerge == "only counts"):
        df.to_csv(f"./outputs/collection/{runName}/merged/countTable_{biomName}.tsv",sep="\t",index=False)
    if(rankToMerge < 10):
        df2 = df
        df2 = df2.reset_index()
        for index, row in df.iterrows():
            rowRaw = row['#KEY']
            rowSplit = rowRaw.split(";")
            if(len(rowSplit) >= rankToMerge):
                df2.loc[index,"#KEY"] = rowSplit[rankToMerge-1].replace("__","_taxa_")
            else:
                df2.loc[index,"#KEY"] = "noValue"

        df2 = df2[df2["#KEY"] != "noValue"]
        df2 = df2.groupby(["#KEY"]).sum()
        df2 = df2.reset_index()
        df2 = df2.drop(columns=["index"])
        df2.to_csv(f"./outputs/collection/{runName}/merged/mergedOn_{galaxyInput_rankToMerge}_{biomName}.tsv",sep="\t",index=False)
    if(galaxyInput_rankToMerge == "all"):
        tables = []
        for i in range(1,9):
            df2 = df
            df2 = df2.reset_index()
            for index, row in df.iterrows():
                rowRaw = row['#KEY']
                rowSplit = rowRaw.split(";")
                if(len(rowSplit) >= i):
                    df2.loc[index,"#KEY"] = rowSplit[i-1].replace("__","_taxa_")
                else:
                    df2.loc[index,"#KEY"] = "noValue"
            df2 = df2[df2["#KEY"] != "noValue"]
            df2 = df2.groupby(["#KEY"]).sum()
            df2 = df2.reset_index()
            df2 = df2.drop(columns=["index"])
            tables.append(df2)
        result = pd.concat(tables)
        result.to_csv(f"./outputs/collection/{runName}/merged/mergedOn_{galaxyInput_rankToMerge}_{biomName}.tsv",sep="\t",index=False)


In [153]:
countFoldersForRun = os.listdir(f"./outputs/collection/{runName}")
for biom in inputBioms:
    print(biomNameMapping[biom])
    for taxa in ["superkingdom","kingdom","phylum","class","order","family","genus","species","all","only counts"]:
        MergeOnTaxaLevelFunction(taxa,runName,biomNameMapping[biom])

biom1
biom2


In [42]:
######################################
# Decalration of Normalization Methods
######################################
from time import sleep

def Normalize_Softmax(_inputDF):
    # works same as in galaxy
    for column in _inputDF:
        if(column != "#KEY"):
            _inputDF[column] = _inputDF[column] / np.sum(_inputDF[column])
            _inputDF[column] = _inputDF[column] * 100
    for column in _inputDF:
        if(column != "#KEY"):
            _inputDF[column] = np.exp(_inputDF[column]) / np.sum(np.exp(_inputDF[column]))
    return _inputDF

def Normalize_RelativeAbundance(_inputDF):
    for column in _inputDF:
        if(column != "#KEY"):
            _inputDF[column] = _inputDF[column] / np.sum(_inputDF[column])
    return _inputDF

def Normalize_Sigmoid(_inputDF):
    for column in _inputDF:
        if(column != "#KEY"):
            _inputDF[column] = _inputDF[column] / np.sum(_inputDF[column])
    for column in _inputDF:
        if(column != "#KEY"):
            _inputDF[column] = (((1 / (1 + np.exp(_inputDF[column]))) - 0.5) * 2)
    return _inputDF


import numpy as np
import subprocess

def Normalize_CSS(_inputDF):
    # read table
    header = _inputDF.columns
    index = _inputDF.index
    # save table without index

    _inputDF.to_csv(f"./RData/table",sep="\t",index=False,float_format='{:.6f}'.format,header=False)

    subprocess.call("Rscript test.R",shell=True)
    
    # reload table
    _inputDF = pd.read_csv("RData/table_out",sep="\t",header=0)
    _inputDF.columns = header
    _inputDF.index = index
    return _inputDF

    #return normalized_df

def Normalize_TMM_CONORM(_inputDF):
    return conorm.tmm(_inputDF)

def Normalize_TMM_Galaxy(_inputDF):
    _inputDF = _inputDF.rename_axis('GeneID')

    #save as count Table
    _inputDF.to_csv(f"./RData/count_table",sep="\t",index=True,float_format='{:.6f}'.format,header=True)

    # create factorFile
    fileNames = list(_inputDF.columns.values)
    factors = []
    for i in range(len(fileNames)):
        if(i % 2 == 0):
            factors.append("WT")
        else:
            factors.append("Mut")
    
    _inputDF = pd.DataFrame({"Sample":fileNames, "Genotype":factors})
    _inputDF.to_csv(f"./RData/factor_table",sep="\t",index=False,float_format='{:.6f}'.format,header=True)
    subprocess.call(f"Rscript limma_voom.R -R output\out -o output -m RData\\count_table -f RData\\factor_table -D Mut,WT -x -l 0.0 -p 0.05 -d BH -G 10 -n TMM")
    _inputDF = pd.read_csv('output\\limma-voom_normcounts',sep="\t", header=0,index_col="GeneID")
    _inputDF = _inputDF.rename_axis('#KEY')

    return _inputDF

def Normalize_RLE_Galaxy(_inputDF):
    _inputDF = _inputDF.rename_axis('GeneID')
    _inputDF = _inputDF + 1
    #save as count Table
    _inputDF.to_csv(f"./RData/count_table",sep="\t",index=True,float_format='{:.6f}'.format,header=True)
    
    # add 1 to all counts, to prevent RLE to result in NA, beacause it is based on log
   
    # create factorFile
    fileNames = list(_inputDF.columns.values)
    factors = []
    for i in range(len(fileNames)):
        if(i % 2 == 0):
            factors.append("WT")
        else:
            factors.append("Mut")
    
    _inputDF = pd.DataFrame({"Sample":fileNames, "Genotype":factors})
    _inputDF.to_csv(f"./RData/factor_table",sep="\t",index=False,float_format='{:.6f}'.format,header=True)
    subprocess.call(f"Rscript limma_voom.R -R output\out -o output -m RData\\count_table -f RData\\factor_table -D Mut,WT -x -l 0.0 -p 0.05 -d BH -G 10 -n RLE")
    _inputDF = pd.read_csv('output\\limma-voom_normcounts',sep="\t", header=0,index_col="GeneID")
    _inputDF = _inputDF.rename_axis('#KEY')

    return _inputDF


In [44]:
from time import sleep
######################
# Normalize all Tables
######################
#pd.options.display.float_format = '{:,.6f}'.format
if(not os.path.isdir(f"./outputs/collection/{runName}/normalized/")):
    os.mkdir(f"./outputs/collection/{runName}/normalized/")

tablesToBeNormalized = os.listdir("outputs\\collection\\run_01\\merged")

failedList = []
successList = []

goodList = []

exeptions = []

for table in tablesToBeNormalized:
    print(table)
    # 
    for method in ["Normalize_Softmax","Normalize_RelativeAbundance","Normalize_CSS","Normalize_TMM_Galaxy","Normalize_RLE_Galaxy"]:
        #try:
            print(method)
            filelist = []
            #delete files from other functions
            for directory in ["RData","output"]:
                filelist = [ f for f in os.listdir(directory)]
                for f in filelist:
                    os.remove(os.path.join(directory, f))
            df = pd.read_csv(f'outputs\collection\\run_01\\merged\\{table}',sep="\t", header=0,index_col="#KEY")
            # go through all columns and remove those with 0 counts
            for c in df.columns:
                if(df[c].sum() == 0):
                    print(f"removed column {c}")
                    df = df.drop(columns=c)
            func = globals()[method]
            df = func(df)
            df.to_csv(f"./outputs/collection/{runName}/normalized/{table}_{method}.tsv",sep="\t",float_format='{:.6f}'.format)
            selfBiom = "biom1"
            pair="biom2"
            if("biom2" in table):
                pair = "biom1"
                selfBiom = "biom2"
            
            if(f'{method}_{table.replace(selfBiom,pair)}' in successList):
                goodList.append(f'{method}_{table.replace(selfBiom,"Both")}')

            successList.append(f'{method}_{table}')

        #except Exception as e:
            #exeptions.append(e)
            #failedList.append(f'{method}_{table}')
goodList

countTable_biom1.tsv
Normalize_Softmax
Normalize_RelativeAbundance
Normalize_CSS
Normalize_TMM_Galaxy
Normalize_RLE_Galaxy
countTable_biom2.tsv
Normalize_Softmax
Normalize_RelativeAbundance
Normalize_CSS
Normalize_TMM_Galaxy
Normalize_RLE_Galaxy
mergedOn_all_biom1.tsv
Normalize_Softmax
Normalize_RelativeAbundance
Normalize_CSS
Normalize_TMM_Galaxy
Normalize_RLE_Galaxy
mergedOn_all_biom2.tsv
Normalize_Softmax
Normalize_RelativeAbundance
Normalize_CSS
Normalize_TMM_Galaxy
Normalize_RLE_Galaxy
mergedOn_class_biom1.tsv
Normalize_Softmax
removed column MGYA00653397-SRS5061631
Normalize_RelativeAbundance
removed column MGYA00653397-SRS5061631
Normalize_CSS
removed column MGYA00653397-SRS5061631
Normalize_TMM_Galaxy
removed column MGYA00653397-SRS5061631
Normalize_RLE_Galaxy
removed column MGYA00653397-SRS5061631
mergedOn_class_biom2.tsv
Normalize_Softmax
Normalize_RelativeAbundance
Normalize_CSS
Normalize_TMM_Galaxy
Normalize_RLE_Galaxy
mergedOn_family_biom1.tsv
Normalize_Softmax
removed col

['Normalize_Softmax_countTable_Both.tsv',
 'Normalize_RelativeAbundance_countTable_Both.tsv',
 'Normalize_CSS_countTable_Both.tsv',
 'Normalize_TMM_Galaxy_countTable_Both.tsv',
 'Normalize_RLE_Galaxy_countTable_Both.tsv',
 'Normalize_Softmax_mergedOn_all_Both.tsv',
 'Normalize_RelativeAbundance_mergedOn_all_Both.tsv',
 'Normalize_CSS_mergedOn_all_Both.tsv',
 'Normalize_TMM_Galaxy_mergedOn_all_Both.tsv',
 'Normalize_RLE_Galaxy_mergedOn_all_Both.tsv',
 'Normalize_Softmax_mergedOn_class_Both.tsv',
 'Normalize_RelativeAbundance_mergedOn_class_Both.tsv',
 'Normalize_CSS_mergedOn_class_Both.tsv',
 'Normalize_TMM_Galaxy_mergedOn_class_Both.tsv',
 'Normalize_RLE_Galaxy_mergedOn_class_Both.tsv',
 'Normalize_Softmax_mergedOn_family_Both.tsv',
 'Normalize_RelativeAbundance_mergedOn_family_Both.tsv',
 'Normalize_CSS_mergedOn_family_Both.tsv',
 'Normalize_TMM_Galaxy_mergedOn_family_Both.tsv',
 'Normalize_RLE_Galaxy_mergedOn_family_Both.tsv',
 'Normalize_Softmax_mergedOn_genus_Both.tsv',
 'Normalize

In [45]:
len(goodList)

50

In [56]:
if(not os.path.isdir(f"./outputs/collection/{runName}/deepMicroInput/")):
    os.mkdir(f"./outputs/collection/{runName}/deepMicroInput/")

pairs = {}
pairsNames = {}
for merge in ["superkingdom","kingdom","phylum","class","order","family","genus","species","all","only counts"]:
    fileNameStart = f"mergedOn_{merge}_"
    if(merge == "only counts"):
        fileNameStart = f"countTable_"
    for normalizationMethod in ["Normalize_Softmax","Normalize_RelativeAbundance","Normalize_CSS","Normalize_TMM_Galaxy","Normalize_RLE_Galaxy"]:
        pair = []
        pairNames = []
        for biom in inputBioms:
            biomName = biomNameMapping[biom]
            fileName = f'{fileNameStart}{biomName}.tsv_{normalizationMethod}.tsv'
            if(os.path.exists(f'outputs/collection/{runName}/normalized/{fileName}')):
                #print(f'outputs/collection/{runName}/normalized/{fileName}')
                df = pd.read_csv(f'outputs/collection/{runName}/normalized/{fileName}',sep="\t",header=0,index_col="#KEY")
                pair.append(df)
                pairNames.append(biomName)
        pairs[f'{fileNameStart}_{normalizationMethod}'] = pair
        pairsNames[f'{fileNameStart}_{normalizationMethod}'] = pairNames


In [92]:
from sklearn.preprocessing import LabelEncoder



for pair in pairs.keys():
    le = LabelEncoder()
    samlesToBiomMapping = {}
    dfList = []
    i = 0
    for df in pairs[pair]:
        df = df.rename_axis('phylum')
        df = df.reset_index()
        df = df.drop(df[df['phylum'] == 'Unassigned'].index)
        df = df.set_index('phylum')
        for c in df.columns:
            samlesToBiomMapping[c] = pairsNames[pair][i]
        dfList.append(df)
        i += 1
    df = dfList[0].join(dfList[1],on='phylum',how="outer")                          ############################################# add outer join
    df = df.fillna(0)
    # remove samples without counts
    for c in df.columns:
        if(df[c].sum() == 0):
            print(f"removed column {c} from {pair}")
            df = df.drop(columns=c)
    df = df.transpose()
    df.to_csv(f"./outputs/collection/{runName}/deepMicroInput/{pair}_X",sep=",",header=False,index=False,float_format='{:.6f}'.format)
    df = df.reset_index()
    df = df.rename(columns={"index":"phylum"})
    df = df.replace({"phylum": samlesToBiomMapping})
    df = df["phylum"]
    le.fit(df)
    #df["phylum"] = list(le.transform(df))
    df = pd.DataFrame(list(le.transform(df)))
    df.to_csv(f"./outputs/collection/{runName}/deepMicroInput/{pair}_Y",sep=",",header=False,index=False,float_format='{:.6f}'.format)
df

removed column MGYA00160006-DRS049568 from mergedOn_species__Normalize_Softmax
removed column MGYA00185679-ERS2517172 from mergedOn_species__Normalize_Softmax
removed column MGYA00185696-ERS2517202 from mergedOn_species__Normalize_Softmax


,0
0,0
1,0
2,0
3,0
4,0
...,...
969,1
970,1
971,1
972,1


0      biom1
1      biom1
2      biom1
3      biom1
4      biom1
       ...  
969    biom2
970    biom2
971    biom2
972    biom2
973    biom2
Name: phylum, Length: 974, dtype: object